## Aggregate files where necessary

In [16]:
from pathlib import Path
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq
from omegaconf import OmegaConf
from tqdm import tqdm

In [14]:
config = OmegaConf.load("config.yaml")

filename = f"{config.output.path}citing.parquet"
chunk_size = 100_000
# destination = "data/citing_aggregated.parquet"

In [28]:
def aggregate(query, tbl_src, name_suffix_out, chunk_size=100_000):
    devrun_maxiter = 10

    query = f"{query} FROM {tbl_src!s}"
    destination = f"{tbl_src.stem}{name_suffix_out}"
    destination = Path(destination).with_suffix(tbl_src.suffix)

    with duckdb.connect() as con:
        print(query)
        arrow_stream = con.execute(query).fetch_record_batch(rows_per_batch=chunk_size)

        first_batch = next(arrow_stream)
        with pa.parquet.ParquetWriter(destination, first_batch.schema) as writer:
            writer.write_batch(first_batch)
            for i, batch in tqdm(enumerate(arrow_stream), desc="Processing records"):
                writer.write_batch(batch)
                if i > devrun_maxiter:
                    break

In [29]:
for tbl, query in config.aggregate.tables.items():
    tbl_src = (Path(config.output.path) / Path(tbl)).with_suffix(config.output.extension)
    aggregate(query, tbl_src, config.aggregate.suffix)


SELECT DISTINCT AuthorId
, Year
, CitedAuthorId
 FROM data/citing.parquet


ParserException: Parser Error: syntax error at or near "/"

In [22]:
config.aggregate.suffix

'_aggregated'

In [27]:
f"{Path("/path/myfile.txt").stem}_test"

'myfile_test'

In [8]:
config.aggregate.tables

{'citing': 'SELECT DISTINCT AuthorId\n, Year\n, CitedAuthorId\n'}

In [109]:
config.db.views.keys()
config.output.path

'data/'

In [5]:
config.aggregate.tables

['citing']

In [113]:
query = f"""
        SELECT DISTINCT AuthorId
            , Year
            , CitedAuthorId
        FROM '{filename}'
     """

Processing records: 3275it [01:07, 48.28it/s]
